# Fingerprint Feature Extraction


### Introduction:

Fingerprint biometrics involve: Image Acquisition, Image Enhancing, Feature extraction and matching with template. Since the dataset has unique fingerprints I will implement feature extraction and different techniques for enhancing images such as Edge detection, adaptive thresholding. Feature extraction constitutes of Ridge detection (level 1 feature) and Minutiae extraction (level 3 feature) to generate a template after whicha query image is matched using the metric ROC AUC curve.


### Dependencies and Data

In [1]:
import numpy as np
import glob
import random
import imageio
import PIL, cv2
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from skimage.morphology import convex_hull_image, erosion
from skimage.morphology import square
import matplotlib.image as mpimg
import skimage
import math
from scipy.ndimage.filters import convolve
from PIL import Image,ImageFilter
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
import os

In [2]:
# KAGGLE FINGERPRINT DATA

DATA_DIR = "../SOCOFing/Real/"
list_dirs = list(glob.glob(DATA_DIR+"*.BMP"))
num_images = len(list_dirs)


In [3]:
num_images

6000

### Termination and Bifurcation detection and Minutiae Extraction

The given code extracts features like Termination, Bifurcation and Minutiae from finger prints, the output is shown below the code:


In [4]:
def getTerminationBifurcation(img, mask):
    img = img == 255;
    (rows, cols) = img.shape;
    minutiaeTerm = np.zeros(img.shape);
    minutiaeBif = np.zeros(img.shape);
    
    for i in range(1,rows-1):
        for j in range(1,cols-1):
            if(img[i][j] == 1):
                block = img[i-1:i+2,j-1:j+2];
                block_val = np.sum(block);
                if(block_val == 2):
                    minutiaeTerm[i,j] = 1;
                elif(block_val == 4):
                    minutiaeBif[i,j] = 1;
    
    mask = convex_hull_image(mask>0)
    mask = erosion(mask, square(5))         
    minutiaeTerm = np.uint8(mask)*minutiaeTerm
    return(minutiaeTerm, minutiaeBif)

In [5]:
class MinutiaeFeature(object):
    def __init__(self, locX, locY, Orientation, Type):
        self.locX = locX;
        self.locY = locY;
        self.Orientation = Orientation;
        self.Type = Type;

def computeAngle(block, minutiaeType):
    angle = 0
    (blkRows, blkCols) = np.shape(block);
    CenterX, CenterY = (blkRows-1)/2, (blkCols-1)/2
    if(minutiaeType.lower() == 'termination'):
        sumVal = 0;
        for i in range(blkRows):
            for j in range(blkCols):
                if((i == 0 or i == blkRows-1 or j == 0 or j == blkCols-1) and block[i][j] != 0):
                    angle = -math.degrees(math.atan2(i-CenterY, j-CenterX))
                    sumVal += 1
                    if(sumVal > 1):
                        angle = float('nan');
        return(angle)
    elif(minutiaeType.lower() == 'bifurcation'):
        (blkRows, blkCols) = np.shape(block);
        CenterX, CenterY = (blkRows - 1) / 2, (blkCols - 1) / 2
        angle = []
        sumVal = 0;
        for i in range(blkRows):
            for j in range(blkCols):
                if ((i == 0 or i == blkRows - 1 or j == 0 or j == blkCols - 1) and block[i][j] != 0):
                    angle.append(-math.degrees(math.atan2(i - CenterY, j - CenterX)))
                    sumVal += 1
        if(sumVal != 3):
            angle = float('nan')
        return(angle)


def extractMinutiaeFeatures(skel, minutiaeTerm, minutiaeBif):
    FeaturesTerm = []

    minutiaeTerm = skimage.measure.label(minutiaeTerm, connectivity=2);
    RP = skimage.measure.regionprops(minutiaeTerm)
    
    WindowSize = 2          
    FeaturesTerm = []
    for i in RP:
        (row, col) = np.int16(np.round(i['Centroid']))
        block = skel[row-WindowSize:row+WindowSize+1, col-WindowSize:col+WindowSize+1]
        angle = computeAngle(block, 'Termination')
        FeaturesTerm.append(MinutiaeFeature(row, col, angle, 'Termination'))

    FeaturesBif = []
    minutiaeBif = skimage.measure.label(minutiaeBif, connectivity=2);
    RP = skimage.measure.regionprops(minutiaeBif)
    WindowSize = 1 
    for i in RP:
        (row, col) = np.int16(np.round(i['Centroid']))
        block = skel[row-WindowSize:row+WindowSize+1, col-WindowSize:col+WindowSize+1]
        angle = computeAngle(block, 'Bifurcation')
        FeaturesBif.append(MinutiaeFeature(row, col, angle, 'Bifurcation'))
    return(FeaturesTerm, FeaturesBif)

def ShowResults(skel, TermLabel, BifLabel):
    minutiaeBif = TermLabel * 0;
    minutiaeTerm = BifLabel * 0;

    (rows, cols) = skel.shape
    DispImg = np.zeros((rows, cols, 3), np.uint8)
    DispImg[:, :, 0] = skel;
    DispImg[:, :, 1] = skel;
    DispImg[:, :, 2] = skel;

    RP = skimage.measure.regionprops(BifLabel)
    for idx, i in enumerate(RP):
        (row, col) = np.int16(np.round(i['Centroid']))
        minutiaeBif[row, col] = 1;
        (rr, cc) = skimage.draw.circle_perimeter(row, col, 1);
        skimage.draw.set_color(DispImg, (rr, cc), (255, 0, 0));

    RP = skimage.measure.regionprops(TermLabel)
    for idx, i in enumerate(RP):
        (row, col) = np.int16(np.round(i['Centroid']))
        minutiaeTerm[row, col] = 1;
        (rr, cc) = skimage.draw.circle_perimeter(row, col, 1);
        skimage.draw.set_color(DispImg, (rr, cc), (0, 0, 255));
        
    plt.figure(figsize=(6,6))
    plt.title("Minutiae extraction results")
    plt.imshow(DispImg)

In [6]:

def extract_label(img_path,train = True):
    filename, _ = os.path.splitext(os.path.basename(img_path))

    subject_id, etc = filename.split('__')
    noise = ""
    
    # train data contain what type of noise in the end, ex. 100__M_Left_thumb_finger_Obl.BMP
    if train:
        gender, lr, finger, _, noise = etc.split('_')
    else:
        gender, lr, finger, _ = etc.split('_')
  
    gender = 0 if gender == 'M' else 1
    lr = 0 if lr == 'Left' else 1

    # We want to return which finger it is
    if finger == 'thumb':
        finger = 0
    elif finger == 'index':
        finger = 1
    elif finger == 'middle':
        finger = 2
    elif finger == 'ring':
        finger = 3
    elif finger == 'little':
        finger = 4
    
    return subject_id, noise

In [11]:

# save all the feature from all the finger 
# the database store all the information of the finger print location
# bif_location store the x,y of the feature
# fingerprint_database: axis: number of finger print, axis2: bif_locations, term_locations
# TermLabel: return the position of termination
# BifLabel: return the position of Bifurcation

term_database = []
bif_database = []
id_database = []

index = 1
for file in list_dirs:
    if index%1000 == 0:
        print("Testing image number:", index)
    image = cv2.imread(file, 0)
    image = image[0:94,0:94]
    ret, th = cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) 
    th = th/255
    skeleton = skimage.morphology.skeletonize(th)
    skeleton = np.uint8(skeleton)*255
    mask = image*255

    (minutiaeTerm, minutiaeBif) = getTerminationBifurcation(skeleton, mask)
    FeaturesTerm, FeaturesBif = extractMinutiaeFeatures(skeleton, minutiaeTerm, minutiaeBif)
    BifLabel = skimage.measure.label(minutiaeBif, connectivity=1) >= 1
    BifLabel = BifLabel.astype(int)
    TermLabel = skimage.measure.label(minutiaeTerm, connectivity=1) >= 1
    TermLabel = TermLabel.astype(int)
    
    id_, noise = extract_label(file,train = False)
    id_database.append(id_)
    bif_database.append(BifLabel)
    term_database.append(TermLabel)
    index += 1
    


Testing image number: 1000
Testing image number: 2000
Testing image number: 3000
Testing image number: 4000
Testing image number: 5000
Testing image number: 6000


In [12]:

id_database[0]

'100'

In [13]:
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col))
        gauss = gauss.reshape(row,col)
        noisy = image + gauss
        return noisy

    elif noise_typ == "s&p":
        row,col = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col = image.shape
        gauss = np.random.randn(row,col)
        gauss = gauss.reshape(row,col)        
        noisy = image + image * gauss
        return noisy

In [30]:
# testing image
test_DATA_DIR = "../SOCOFing/Real/"
num_true = 0
num_image_tested = 0
b_d = np.array(bif_database)
t_d = np.array(term_database)

test_list_dirs = list(glob.glob(test_DATA_DIR+"*.BMP"))
test_num_images = len(test_list_dirs)

print(test_num_images)
while num_image_tested <= 100:
    idx = random.randint(0, test_num_images-1)
    print(idx)
    file = test_list_dirs[idx]
    test_id, n = extract_label(file, False)
#     if n != "Obl":
#         continue
        
    print("Testing image number:", num_image_tested)
    print("Preprocessing image")
        
    test_image = cv2.imread(file, 0)
    test_image = test_image[0:94,0:94]
    
    noisy_image = noisy("poisson", test_image)
    noisy_image = noisy_image.astype('uint8')
    test_ret, test_th = cv2.threshold(noisy_image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) 
    test_th = test_th/255
    test_skeleton = skimage.morphology.skeletonize(test_th)
    test_skeleton = np.uint8(test_skeleton)*255
    test_mask = test_image*255

    (minutiaeTerm, minutiaeBif) = getTerminationBifurcation(test_skeleton, test_mask)
    FeaturesTerm, FeaturesBif = extractMinutiaeFeatures(skeleton, minutiaeTerm, minutiaeBif)
    test_BifLabel = skimage.measure.label(minutiaeBif, connectivity=1)  >= 1
    test_BifLabel = test_BifLabel.astype(int)
    test_TermLabel = skimage.measure.label(minutiaeTerm, connectivity=1)  >= 1
    test_TermLabel = test_TermLabel.astype(int)

    
    print("Calculating MSE")
    # calcalculate mse
    error = []
    for i in range(len(b_d)): 
#         if i%100 == 0:
#             print(i)
        mse = (np.square(b_d[i] - test_BifLabel)).mean(axis=0).mean(axis=0)
        mse += (np.square(t_d[i] - test_TermLabel)).mean(axis=0).mean(axis=0)
        error.append(mse)
    
    error = np.array(error)
    print("Real:", id_database[error.argmin()], "Predicted:", test_id)
    if id_database[error.argmin()] == test_id:
        num_true += 1
        
    num_image_tested += 1
    
print("Accuracy:", num_true/num_image_tested*100)
        

6000
3290
Testing image number: 0
Preprocessing image
Calculating MSE
Real: 226 Predicted: 398
833
Testing image number: 1
Preprocessing image
Calculating MSE
Real: 176 Predicted: 176
4404
Testing image number: 2
Preprocessing image
Calculating MSE
Real: 498 Predicted: 498
1689
Testing image number: 3
Preprocessing image
Calculating MSE
Real: 252 Predicted: 252
2106
Testing image number: 4
Preprocessing image
Calculating MSE
Real: 290 Predicted: 290
2334
Testing image number: 5
Preprocessing image
Calculating MSE
Real: 310 Predicted: 310
4706
Testing image number: 6
Preprocessing image
Calculating MSE
Real: 524 Predicted: 524
692
Testing image number: 7
Preprocessing image
Calculating MSE
Real: 163 Predicted: 163
1807
Testing image number: 8
Preprocessing image
Calculating MSE
Real: 263 Predicted: 263
2399
Testing image number: 9
Preprocessing image
Calculating MSE
Real: 316 Predicted: 316
1182
Testing image number: 10
Preprocessing image
Calculating MSE
Real: 207 Predicted: 207
5784
T

Real: 481 Predicted: 481
5342
Testing image number: 91
Preprocessing image
Calculating MSE
Real: 582 Predicted: 582
5956
Testing image number: 92
Preprocessing image
Calculating MSE
Real: 96 Predicted: 96
3570
Testing image number: 93
Preprocessing image
Calculating MSE
Real: 422 Predicted: 422
1091
Testing image number: 94
Preprocessing image
Calculating MSE
Real: 19 Predicted: 19
3770
Testing image number: 95
Preprocessing image
Calculating MSE
Real: 440 Predicted: 440
4667
Testing image number: 96
Preprocessing image
Calculating MSE
Real: 520 Predicted: 520
4572
Testing image number: 97
Preprocessing image
Calculating MSE
Real: 512 Predicted: 512
4636
Testing image number: 98
Preprocessing image
Calculating MSE
Real: 518 Predicted: 518
1059
Testing image number: 99
Preprocessing image
Calculating MSE
Real: 196 Predicted: 196
5023
Testing image number: 100
Preprocessing image
Calculating MSE
Real: 553 Predicted: 553
Accuracy: 94.05940594059405


In [20]:
len(bif_database)

6000

In [46]:
b_d = np.array(bif_database)
error = []

for finger in b_d:
    mse = (np.square(finger - test_BifLabel)).mean(axis=0).mean(axis=0)
    error.append(mse)
    
print(error[0:50])
error = np.array(error)
print("Predicted Id:", id_database[error.argmin()], "Real Id: ", test_id, "Error:", mse)
        
            

[2138.4188546853775, 3032.7235174287002, 2121.3253734721593, 2787.240493435944, 2231.780556813037, 2322.9550701674966, 2123.5546627433228, 2545.0889542779537, 2123.5508148483473, 2682.6616115889547, 3257.9843820733367, 3035.8037573562706, 2536.54889090086, 2128.1412403802624, 8954.983703033045, 2220.500226346763, 2203.3879583521957, 2138.6698732458126, 2140.6461068356716, 2209.913761883205, 2247.9148936170213, 3454.579447713898, 2251.869058397465, 2623.661045722046, 2234.9483929379808, 2229.7858759619735, 2373.56631960163, 2152.002716161159, 2282.2437754640105, 2256.0941602535086, 2697.913082842915, 5345.024219103667, 2665.9367360796737, 2161.9968311453144, 2465.946808510638, 3206.5347442281577, 2524.882752376641, 2157.337030330466, 2163.195224083295, 2424.6140787686736, 6566.993322770483, 2129.6181530104122, 4094.2149162516976, 3047.6708918062473, 2141.711407876867, 2149.2133318243546, 2129.399049343594, 2525.9061792666366, 2428.7556586690807, 2312.13614757809]
Predicted Id: 170 Real 

In [31]:
for ids in id_database:
    print(ids)
    

171
463
470
367
446
349
127
277
308
334
20
26
86
431
434
142
72
417
127
552
589
364
149
342
180
575
444
183
36
232
400
155
240
556
278
303
252
23
315
501
378
50
244
318
351
202
218
386
5
26
554
130
236
262
334
156
71
293
295
272
233
522
583
193
111
296
568
50
111
452
112
247
462
38
296
297
208
119
381
165
320
260
583
84
39
331
461
84
370
325
72
205
343
557
560
264
574
518
141
380
551
236
238
318
543
463
153
18
355
375
137
599
76
79
217
549
124
290
270
403
477
158
78
238
383
590
81
407
436
6
527
191
311
555
201
320
537
170
6
149
328
310
311
24
554
476
522
544
72
113
357
512
271
247
185
39
319
180
270
118
582
319
155
22
357
283
505
332
413
300
331
133
534
22
199
521
579
214
106
388
377
3
255
121
314
461
301
341
248
331
540
84
192
400
427
252
24
164
585
521
409
286
344
245
513
327
591
141
418
184
60
304
34
195
83
418
3
113
450
137
171
33
305
245
391
369
46
177
371
336
366
588
262
91
281
443
375
486
222
459
386
310
328
460
368
30
187
218
599
466
291
535
555
292
212
167
388
215
357
332
577


In [12]:
def featureExtractor(img):
    img = img[0:94,0:94]
    ret, th = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) 
    th = th/255
    skeleton = skimage.morphology.skeletonize(th)
    skeleton = np.uint8(skeleton)*255
    mask = img*255

    (minutiaeTerm, minutiaeBif) = getTerminationBifurcation(skeleton, mask)
    FeaturesTerm, FeaturesBif = extractMinutiaeFeatures(skeleton, minutiaeTerm, minutiaeBif)
    BifLabel = skimage.measure.label(minutiaeBif, connectivity=1)  >= 1
    BifLabel = BifLabel.astype(int)
    TermLabel = skimage.measure.label(minutiaeTerm, connectivity=1)  >= 1
    TermLabel = TermLabel.astype(int)
    
    return BifLabel, TermLabel

In [13]:
img = cv2.imread(list_dirs[0], 0)
BifLabel, TermLabel = featureExtractor(img)

In [16]:
TermLabel.max()

1